In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class DenseBlock(nn.Module):
    def __init__(self, input_num_planes, rate_inc):
        super().__init__()
        self.denceblock = self.Sequential(
              nn.BatchNorm2d(input_num_planes),
              # 연산량을 증가시키지 않으면서 풍푸한 표현을 위해 1x1 으로 1번 과정이 거쳐진다.
              # 이때 rate_inc 가 k growth rate 이다.
              # 4는 논문에서 정한 가장 밸런스 있는 값이다.
              # 채널수를 일정하게 맞춰 줘야 하므로 다시 k 개의 채널로 조정한다.
              nn.Relu(),
              nn.Conv2d(in_channels=input_num_planes, out_channels=4*rate_inc, kernel_size=1, bias=False),
              nn.BatchNorm2d(4*rate_inc),
              # 채널수를 일정하게 맞춰 줘야 하므로 다시 k 개의 채널로 조정한다.
              nn.Conv2d(in_channels=4*rate_inc, out_channels=rate_inc, kernel_size=3, padding=1, bias=False),
        )
    def forward(self, inp):
        op = self.denseblock(inp)
        op = torch.cat([op, inp], 1)  # 합치기
        return op

class TransBlock(nn.Module):
    def __init__(self, input_num_planes, output_num_planes):
        super().__init__()
        self.batch_norm = nn.BatchNorm2d(input_num_planes)
        self.conv_layer = nn.Conv2d(in_channels=input_num_planes, out_channels=output_num_planes, kernel_size=1, bias=False)
    def forward(self, inp):
        op = self.conv_layer(F.relu(self.batch_norm(inp)))
        op = F.avg_pool2d(op, 2)
        return op